In [63]:
import lxml.etree as etree
import subprocess
import tempfile
import time
import psutil
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap
import os

%matplotlib inline
import matplotlib.pyplot as plt
import gdal
import numpy as np
from shapely.wkt import loads
from shapely.geometry import Point
import osr
import ogr
from shapely import geometry

In [64]:


class GraphProcessor():
    
    def __init__(self):
        self.root = etree.Element('graph')
    
        version = etree.SubElement(self.root, 'version')
        version.text = '1.0'
        self.pid = None
        self.p = None
   
    def view_graph(self):
        
        print etree.tostring(self.root , pretty_print=True)
        
    def add_node(self, node_id, operator, parameters, source):
    
        xpath_expr = '/graph/node[@id="%s"]' % node_id

        if len(self.root.xpath(xpath_expr)) != 0:

            node_elem = self.root.xpath(xpath_expr)[0]
            operator_elem = self.root.xpath(xpath_expr + '/operator')[0]
            sources_elem = self.root.xpath(xpath_expr + '/sources')[0]
            parameters_elem = self.root.xpath(xpath_expr + '/parameters')

            for key, value in parameters.iteritems():
                p_elem = self.root.xpath(xpath_expr + '/parameters/%s' % key)[0]
                p_elem.text = value
        else:

            node_elem = etree.SubElement(self.root, 'node')
            operator_elem = etree.SubElement(node_elem, 'operator')
            sources_elem = etree.SubElement(node_elem, 'sources')

            if isinstance(source, list):

                for index, s in enumerate(source):
                    if index == 0:  
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')

                    else: 
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct.%s' % str(index))

                    source_product_elem.attrib['refid'] = s

            elif source != '':
                source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')
                source_product_elem.attrib['refid'] = source

            parameters_elem = etree.SubElement(node_elem, 'parameters')
            parameters_elem.attrib['class'] = 'com.bc.ceres.binding.dom.XppDomElement'

            for key, value in parameters.iteritems():

                parameter_elem = etree.SubElement(parameters_elem, key)
                parameter_elem.text = value

        node_elem.attrib['id'] = node_id

        operator_elem.text = operator 

    def save_graph(self, filename):
        
        with open(filename, 'wb') as file:
            file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
            file.write(etree.tostring(self.root, pretty_print=True))
     
    def plot_graph(self):
        
        for node_id in self.root.xpath('/graph/node/@id'):
            

            xpath_expr = '/graph/node[@id="%s"]' % node_id
            
            if len(self.root.xpath(xpath_expr + '/sources/sourceProduct')) != 0:
                print(self.root.xpath(xpath_expr + '/sources/sourceProduct'))[0].attrib['refid']
                print node_id
            else:
                print node_id
        return True
    
    def run(self):
        
        fd, path = tempfile.mkstemp()
        
        try:
        
            self.save_graph(filename=path)
            options = ['/opt/snap/bin/gpt',
               '-x',
               '-c',
               '2048M',
               path]

            p = subprocess.Popen(options,
                stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)

            print p.pid
            res, err = p.communicate()
            print res, err
        finally:
            os.remove(path)
        
def get_snap_parameters(operator):
    
    op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

    op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

    return op_params

In [65]:
mygraph = GraphProcessor()

In [66]:
operator = 'Read'

parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        parameters[param.getName()] = '/data2/S1B_EW_GRDM_1SDH_20180822T114138_20180822T114238_012375_016D07_E770.zip'    
    else:
        parameters[param.getName()] = param.getDefaultValue()
    
node_id = 'Read'
    

mygraph.add_node(node_id, 'Read', parameters, '')

In [67]:
operator = 'ThermalNoiseRemoval'

node_id = 'ThermalNoiseRemoval' 

source_node = 'Read'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('selectedPolarisations', None)
('removeThermalNoise', 'true')
('reIntroduceThermalNoise', 'false')


In [68]:
mygraph.add_node(node_id, operator, parameters, source_node)

In [69]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File' 

source_node = 'ThermalNoiseRemoval'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('orbitType', 'Sentinel Precise (Auto Download)')
('polyDegree', '3')
('continueOnFail', 'false')


In [70]:
mygraph.add_node(node_id, operator, parameters, source_node)

In [71]:
operator = 'Calibration'

node_id = 'Calibration' 

source_node = 'Apply-Orbit-File'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('sourceBandNames', None)
('auxFile', 'Latest Auxiliary File')
('externalAuxFile', None)
('outputImageInComplex', 'false')
('outputImageScaleInDb', 'false')
('createGammaBand', 'false')
('createBetaBand', 'false')
('selectedPolarisations', None)
('outputSigmaBand', 'true')
('outputGammaBand', 'false')
('outputBetaBand', 'false')


In [72]:
mygraph.add_node(node_id, operator, parameters, source_node)

In [73]:
operator = 'Speckle-Filter'

node_id = 'Speckle-Filter' 

source_node = 'Calibration'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('sourceBandNames', None)
('filter', 'Lee Sigma')
('filterSizeX', '3')
('filterSizeY', '3')
('dampingFactor', '2')
('estimateENL', 'false')
('enl', '1.0')
('numLooksStr', '1')
('windowSize', '7x7')
('targetWindowSizeStr', '3x3')
('sigmaStr', '0.9')
('anSize', '50')


In [74]:
mygraph.add_node(node_id, operator, parameters, source_node)

In [75]:
operator = 'Multilook'

node_id = 'Multilook' 

source_node = 'Speckle-Filter'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('sourceBandNames', None)
('nRgLooks', '1')
('nAzLooks', '1')
('outputIntensity', 'false')
('grSquarePixel', 'true')


In [76]:
mygraph.add_node(node_id, operator, parameters, source_node)

In [77]:
operator = 'LinearToFromdB'

node_id = 'LinearToFromdB' 

source_node = 'Multilook'

parameters = dict()

for param in get_snap_parameters(operator):
    
    parameters[param.getName()] = param.getDefaultValue()

In [78]:
mygraph.add_node(node_id, operator, parameters, source_node)

In [79]:
operator = 'Terrain-Correction'

node_id = 'Terrain-Correction' 

source_node = 'LinearToFromdB'

parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'demName':
        parameters[param.getName()] = 'ACE30'    
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [80]:
mygraph.add_node(node_id, operator, parameters, source_node)

In [81]:
operator = 'Subset'

node_id = 'Subset' 

source_node = 'LinearToFromdB'

parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'geoRegion':
        parameters[param.getName()] = 'POLYGON ((-36.2310862509672 83.2595511507302, -29.4071545191827 83.75772664159339, -34.1503429540384 84.5388284015394, -41.4067813117152 83.9752312745992, -36.2310862509672 83.2595511507302))'    
    else:
        parameters[param.getName()] = param.getDefaultValue()
        
parameters

{'bandNames': None,
 'copyMetadata': 'false',
 'fullSwath': 'false',
 'geoRegion': 'POLYGON ((-36.2310862509672 83.2595511507302, -29.4071545191827 83.75772664159339, -34.1503429540384 84.5388284015394, -41.4067813117152 83.9752312745992, -36.2310862509672 83.2595511507302))',
 'region': None,
 'subSamplingX': '1',
 'subSamplingY': '1',
 'tiePointGridNames': None}

In [82]:
mygraph.add_node(operator, 
             node_id, 
             parameters,
             source_node)

In [83]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/data2/S1B_EW_GRDM_1SDH_20180822T114138_20180822T114238_012375_016D07_E770.zip</file>
    </parameters>
  </node>
  <node id="ThermalNoiseRemoval">
    <operator>ThermalNoiseRemoval</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <reIntroduceThermalNoise>false</reIntroduceThermalNoise>
      <selectedPolarisations/>
      <removeThermalNoise>true</removeThermalNoise>
    </parameters>
  </node>
  <node id="Apply-Orbit-File">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="ThermalNoiseRemoval"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>

In [84]:
operator = 'Write'

node_id = 'Write' 

source_node = 'Subset'

parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        
        param_value = 'sigma0'
             
    elif param.getName() == 'formatName':
                
        param_value = 'GeoTIFF-BigTiff'
        
    else:
    
        param_value = param.getDefaultValue()
    
    
    print param.getName(), param_value
    
    parameters[param.getName()] = param_value


file sigma0
formatName GeoTIFF-BigTiff
deleteOutputOnFailure true
writeEntireTileRows true
clearCacheAfterRowWrite false


In [85]:
mygraph.add_node(operator, 
             node_id, 
             parameters,
             source_node) 

In [86]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/data2/S1B_EW_GRDM_1SDH_20180822T114138_20180822T114238_012375_016D07_E770.zip</file>
    </parameters>
  </node>
  <node id="ThermalNoiseRemoval">
    <operator>ThermalNoiseRemoval</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <reIntroduceThermalNoise>false</reIntroduceThermalNoise>
      <selectedPolarisations/>
      <removeThermalNoise>true</removeThermalNoise>
    </parameters>
  </node>
  <node id="Apply-Orbit-File">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="ThermalNoiseRemoval"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>

In [ ]:
mygraph.run()

4349


In [30]:
fig = plt.figure(figsize=(20,20))

ds = gdal.Open('sigma0.tif')
band = ds.GetRasterBand(1)

imgplot = plt.imshow(band.ReadAsArray().astype(np.float),
                         cmap=plt.cm.binary, 
                         vmin=-15, 
                         vmax=5)

plt.show()

In [31]:
poi = loads('POINT (-35.3 83.90000000000001)')

In [32]:
def extend_aoi(center_x, center_y, extent):
    
    center_polar = loads(convert_coords(4326, 3995, Point(center_x, center_y).wkt))
    
    ll = convert_coords(3995, 4326, Point(center_polar.x - extent,  center_polar.y - extent).wkt)
    lr = convert_coords(3995, 4326, Point(center_polar.x + extent,  center_polar.y - extent).wkt)
    ur = convert_coords(3995, 4326, Point(center_polar.x + extent,  center_polar.y + extent).wkt)
    ul = convert_coords(3995, 4326, Point(center_polar.x - extent,  center_polar.y + extent).wkt)


    pointList = [loads(ll),
             loads(lr), 
             loads(ur), 
             loads(ul), 
             loads(ll)]

    extended_aoi = geometry.Polygon([[p.x, p.y] for p in pointList]).wkt
    
    return extended_aoi

In [33]:
def convert_coords(source_epsg, target_epsg, geom):

    source = osr.SpatialReference()
    source.ImportFromEPSG(source_epsg)

    target = osr.SpatialReference()
    target.ImportFromEPSG(target_epsg)

    transform = osr.CoordinateTransformation(source, target)

    point = ogr.CreateGeometryFromWkt(geom)
    point.Transform(transform)

    return point.ExportToWkt()

In [34]:
aoi = loads(extend_aoi(poi.x, poi.y, 50000)).bounds
aoi

extend_aoi(poi.x, poi.y, 50000)

'POLYGON ((-36.2310862509672 83.2595511507302, -29.4071545191827 83.75772664159339, -34.1503429540384 84.5388284015394, -41.4067813117152 83.9752312745992, -36.2310862509672 83.2595511507302))'

In [92]:
#/vsimem/
output = 'a.tif'
ds = gdal.Open('sigma0.tif')
ds = gdal.Translate(output, ds, projWin = [ aoi[0], aoi[3], aoi[2], aoi[1]], projWinSRS = 'EPSG:4326')
ds = None


In [ ]:
ds = gdal.Open('sigma0.tif')
band = ds.GetRasterBand(1)

imgplot = plt.imshow(band.ReadAsArray().astype(np.float),
                         cmap=plt.cm.binary, 
                         vmin=-15, 
                         vmax=5)

plt.show()